In [2]:
import pickle
import numpy as np
from tqdm import tqdm
from functools import partial
import pandas as pd

In [3]:
max_n = 4
total_data = [[], [], [], [], []]
for i in range(1, max_n + 1):
    file_name = 'data_256_cat%s.pkl' %i
    loaded_data = pickle.load(open(file_name, 'rb'))

    _image_data, _sentence_data, _sentence_tags, _hashtag_data, _metadata = loaded_data[1]

    _sentence_tags = np.array(_sentence_tags)
    _hashtag_data = np.array(_hashtag_data)
    
    total_data[0].append(_image_data)
    total_data[1].append(_sentence_data)
    total_data[2].append(_sentence_tags)
    total_data[3].append(_hashtag_data)
    total_data[4].append(_metadata)

In [4]:
_image_data = np.concatenate(total_data[0])
_sentence_data = np.concatenate(total_data[1])
_sentence_tags = np.concatenate(total_data[2])
_hashtag_data = np.concatenate(total_data[3])
_metadata = pd.concat(total_data[4])

In [5]:
np.random.seed(1234)
n_valid = 700
n_train = _image_data.shape[0] - n_valid
n_total = n_train + n_valid
indices = np.random.choice(np.arange(_image_data.shape[0]), n_total, replace=False)
train_idx = indices[:n_train]
valid_idx = indices[n_train:]

In [6]:
image_data = _image_data[train_idx]
sentence_data = _sentence_data[train_idx]
sentence_tags = _sentence_tags[train_idx]
hashtag_data = _hashtag_data[train_idx]
metadata = _metadata.iloc[train_idx]

In [7]:
image_valid = _image_data[valid_idx]
sentence_valid = _sentence_data[valid_idx]
sentence_valid = _sentence_tags[valid_idx]
hashtag_valid = _hashtag_data[valid_idx]
metadata_valid = _metadata.iloc[valid_idx]

In [8]:
print(image_data.shape)
print(sentence_data.shape)
print(sentence_tags.shape)
print(hashtag_data.shape)
print(metadata.shape)

(27300, 256, 256, 3)
(27300,)
(27300,)
(27300,)
(27300, 15)


In [85]:
output_chars = set()
for s in sentence_data:
    for c in s:
        if c not in output_chars:
            output_chars.add(c)
            
output_chars = sorted(list(output_chars))
output_chars[0], output_chars[1] = output_chars[1], output_chars[0]
output_chars.append('<UNK>')
output_dict = {c: i for i, c in enumerate(output_chars)}
dict_len = len(output_dict)
max_seq_length = max(map(len, sentence_data))

In [87]:
dec_input_data = np.zeros(shape=(len(sentence_data), max_seq_length, dict_len), dtype='float32')
dec_output_data = np.zeros(shape=(len(sentence_data), max_seq_length), dtype='int32')

for i, s in enumerate(sentence_data):
    for j, c in enumerate(s):
        dec_input_data[i, j, output_dict[c]] = 1.
        if j > 0:
            dec_output_data[i, j-1] = output_dict[c]
            
dec_input_valid = np.zeros(shape=(len(sentence_valid), max_seq_length, dict_len), dtype='float32')
dec_output_valid = np.zeros(shape=(len(sentence_valid), max_seq_length), dtype='int32')
dec_input_valid[:,0,output_dict['\t']] = 1.

for i, s in enumerate(sentence_valid):
    for j, c in enumerate(s):
        if j > 0:
            dec_output_data[i, j-1] = output_dict[c if c in output_dict else '<UNK>']

In [89]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [90]:
tf.reset_default_graph()

learning_rate1 = 0.001
learning_rate2 = 0.001
n_hidden = 128

inputs = tf.placeholder(tf.float32, shape=[None, 256, 256, 3])
with tf.variable_scope('auto_encoder'):
    net = tf.layers.conv2d(inputs, 32, [5,5], strides=2, padding='SAME')
    net = tf.layers.conv2d(net, 8, [5,5], strides=2, padding='SAME')
    middle = tf.layers.conv2d(net, 4, [5,5], strides=2, padding='SAME')
    net = tf.layers.conv2d_transpose(middle, 8, [5,5], strides=2, padding='SAME')
    net = tf.layers.conv2d_transpose(net, 32, [5,5], strides=2, padding='SAME')
    outputs = tf.layers.conv2d_transpose(net, 3, [5,5], strides=2, padding='SAME')

loss = tf.reduce_mean(tf.square(outputs - inputs))
train_op = tf.train.AdamOptimizer(learning_rate1).minimize(loss)

dec_inputs = tf.placeholder(tf.float32, shape=[None, max_seq_length, dict_len])
dec_outputs = tf.placeholder(tf.int32, shape=[None, max_seq_length])

with tf.variable_scope('generation_enc'):
    gen_inputs = tf.layers.dense(middle, 1, activation=None)
    gen_inputs = tf.reshape(gen_inputs, [-1, gen_inputs.shape[1], gen_inputs.shape[2]])
    enc_cell = tf.nn.rnn_cell.GRUCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.8)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, gen_inputs, dtype=tf.float32)

with tf.variable_scope('generation_dec'):
    dec_cell = tf.nn.rnn_cell.GRUCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.8)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_inputs, 
                                            initial_state=enc_states,
                                            dtype=tf.float32)
    
    outputs2 = tf.layers.dense(outputs, dict_len, activation=None)

loss2 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = outputs2, labels = dec_outputs))
var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generation*')
train_op2 = tf.train.AdamOptimizer(learning_rate2).minimize(loss2, var_list=var_list)
train_op2_fine = tf.train.AdamOptimizer(learning_rate2).minimize(loss2)

In [93]:
with tf.device('/gpu:0'):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, 
                                            log_device_placement=True))
    sess.run(tf.global_variables_initializer())

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:68:00.0, compute capability: 7.5



In [94]:
batch_size = 700
n_epoch = 20
n_batch = n_train // batch_size

for epoch in range(n_epoch):
    total_loss = 0
    for batch in tqdm(range(n_batch)):
        batch_img = image_data[batch*batch_size:(batch+1)*batch_size]
        _, c = sess.run([train_op, loss], feed_dict={inputs:batch_img})
        total_loss += c
        
    c = sess.run(loss, feed_dict={inputs:image_valid})
    
    print('[Epoch: %s] cost = %.3f, valid cost = %.3f' %(epoch+1, total_loss / n_batch, c))

  0%|          | 0/39 [00:00<?, ?it/s]

[Epoch: 1] cost = 7110.023, valid cost = 2183.689


  0%|          | 0/39 [00:00<?, ?it/s]

[Epoch: 2] cost = 1704.653, valid cost = 1486.916


  0%|          | 0/39 [00:00<?, ?it/s]

...

[Epoch: 18] cost = 463.397, valid cost = 498.658


  0%|          | 0/39 [00:00<?, ?it/s]

[Epoch: 19] cost = 455.204, valid cost = 491.347


100%|██████████| 39/39 [00:28<00:00,  1.37it/s]


[Epoch: 20] cost = 448.639, valid cost = 497.377


In [95]:
batch_size = 700
n_epoch = 30
n_batch = n_train // batch_size

for epoch in range(n_epoch):
    total_loss = 0
    for batch in tqdm(range(n_batch)):
        batch_img = image_data[batch*batch_size:(batch+1)*batch_size]
        batch_dec_in = dec_input_data[batch*batch_size:(batch+1)*batch_size]
        batch_dec_out = dec_output_data[batch*batch_size:(batch+1)*batch_size]
        _, c = sess.run([train_op2, loss2], feed_dict={inputs:batch_img, 
                                                       dec_inputs:batch_dec_in,
                                                       dec_outputs:batch_dec_out})
        total_loss += c
    
    c = sess.run(loss2, feed_dict={inputs:image_valid, 
                                   dec_inputs:dec_input_valid,
                                   dec_outputs:dec_output_valid})
    print('[Epoch %s] cost = %.3f, valid cost = %.3f' %(epoch+1, total_loss / n_batch, c))

  0%|          | 0/39 [00:00<?, ?it/s]

[Epoch 1] cost = 4.515, valid cost = 0.157


  0%|          | 0/39 [00:00<?, ?it/s]

[Epoch 2] cost = 1.304, valid cost = 0.291


  0%|          | 0/39 [00:00<?, ?it/s]

...

[Epoch 33] cost = 0.762, valid cost = 0.054


  0%|          | 0/39 [00:00<?, ?it/s]

[Epoch 34] cost = 0.756, valid cost = 0.054


 18%|█▊        | 7/39 [00:11<00:51,  1.61s/it]


KeyboardInterrupt: 

In [96]:
batch_size = 700
n_epoch = 100
n_batch = n_train // batch_size

for epoch in range(n_epoch):
    total_loss = 0
    for batch in tqdm(range(n_batch)):
        batch_img = image_data[batch*batch_size:(batch+1)*batch_size]
        batch_dec_in = dec_input_data[batch*batch_size:(batch+1)*batch_size]
        batch_dec_out = dec_output_data[batch*batch_size:(batch+1)*batch_size]
        _, c = sess.run([train_op2_fine, loss2], feed_dict={inputs:batch_img, 
                                                       dec_inputs:batch_dec_in,
                                                       dec_outputs:batch_dec_out})
        total_loss += c
    
    c = sess.run(loss2, feed_dict={inputs:image_valid, 
                                   dec_inputs:dec_input_valid,
                                   dec_outputs:dec_output_valid})
    print('[Epoch %s] cost = %.3f, valid cost = %.3f' %(epoch+1, total_loss / n_batch, c))

  0%|          | 0/39 [00:00<?, ?it/s]

[Epoch 1] cost = 0.751, valid cost = 0.053


  0%|          | 0/39 [00:00<?, ?it/s]

[Epoch 2] cost = 0.740, valid cost = 0.052


  0%|          | 0/39 [00:00<?, ?it/s]

...

[Epoch 5] cost = 0.718, valid cost = 0.048


  0%|          | 0/39 [00:00<?, ?it/s]

[Epoch 6] cost = 0.711, valid cost = 0.047


 28%|██▊       | 11/39 [00:18<00:46,  1.65s/it]


KeyboardInterrupt: 

In [79]:
from PIL import Image
import matplotlib.pyplot as plt

In [98]:
def translate(image):
    prediction = tf.argmax(outputs2, 2)
    dec_inp = np.zeros(shape=(1, max_seq_length, dict_len), dtype='float32')
    dec_inp[0,:,output_dict['\t']] = 1.
    # dec_out = np.eye(dict_len)[dec_out]
    result = sess.run(prediction, 
                      feed_dict={inputs: [image],
                                 dec_inputs: dec_inp})
    decoded = [output_chars[i] for i in result[0]]
    end = decoded.index('\n') if '\n' in decoded else len(decoded)
    translated = ''.join(decoded[:end])
    return translated

In [103]:
translate(image_data[50])

'오<UNK><UNK>'

In [80]:
start_idx, end_idx = 50, 100

results = sess.run(outputs2, feed_dict={inputs:image_data[start_idx:end_idx]})
print('정답 예측')

for l, p, img in zip(y_train[start_idx:end_idx], l_label[np.argmax(results, axis=1)], image_data[start_idx:end_idx]):
    print([l==p, l, p])
    plt.imshow(img)
    plt.show()

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [?,256,2069]
	 [[node Placeholder_1 (defined at <ipython-input-70-42deb1a72a5c>:19) ]]
	 [[generation_dec/dense/BiasAdd/_167]]
  (1) Invalid argument: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [?,256,2069]
	 [[node Placeholder_1 (defined at <ipython-input-70-42deb1a72a5c>:19) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'Placeholder_1':
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/svclaw2000/venv_capstone/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/svclaw2000/venv_capstone/lib/python3.7/site-packages/traitlets/config/application.

In [52]:
start_idx, end_idx = 0, 50

results = sess.run(outputs2, feed_dict={inputs:image_valid[start_idx:end_idx]})
print('정답 예측')
[[l==p, l, p] for l, p in zip(y_train[start_idx:end_idx], l_label[np.argmax(results, axis=1)])]

정답 예측


[[False, '애완동물', '여행'],
 [False, '일상', '여행'],
 [False, '애완동물', '여행'],
 [False, '셀카', '패션'],
 [False, '여행', '셀카'],
 [False, '여행', '연애'],
 [False, '애완동물', '음식'],
 [False, '음식', '여행'],
 [False, '패션', '여행'],
 [True, '애완동물', '애완동물'],
 [False, '일상', '애완동물'],
 [False, '여행', '애완동물'],
 [False, '패션', '연애'],
 [True, '패션', '패션'],
 [False, '음식', '여행'],
 [False, '여행', '음식'],
 [False, '음식', '여행'],
 [False, '패션', '애완동물'],
 [False, '음식', '셀카'],
 [False, '애완동물', '연애'],
 [False, '일상', '음식'],
 [True, '여행', '여행'],
 [False, '음식', '패션'],
 [False, '연애', '셀카'],
 [False, '셀카', '음식'],
 [False, '음식', '패션'],
 [False, '애완동물', '연애'],
 [False, '패션', '일상'],
 [False, '셀카', '애완동물'],
 [False, '일상', '패션'],
 [False, '음식', '여행'],
 [True, '패션', '패션'],
 [False, '패션', '일상'],
 [False, '일상', '연애'],
 [True, '연애', '연애'],
 [False, '애완동물', '패션'],
 [False, '일상', '연애'],
 [False, '애완동물', '패션'],
 [False, '애완동물', '패션'],
 [False, '연애', '여행'],
 [False, '셀카', '여행'],
 [True, '음식', '음식'],
 [False, '여행', '애완동물'],
 [True, '애완동물', '애완동물'],
 [Fal